In [5]:
import networkx as nx
import numpy as np
import pickle
import os
from collections import defaultdict
from scipy.sparse.linalg import eigsh, eigs
import torch
import random
from sklearn.model_selection import train_test_split,KFold
import torch_geometric
from torch.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing,GraphSAGE,GlobalAttention
from torch_geometric.utils import degree,add_self_loops
import torch.nn.functional as F
from torch_geometric.nn.pool import global_mean_pool
from torch_geometric.data import HeteroData, DataLoader,Data, Batch
from torch_geometric.loader import NeighborLoader, ImbalancedSampler, DataLoader
import pandas as pd
from torch_geometric.nn.pool import global_mean_pool
from torch_geometric.nn import summary
from scipy.stats import hypergeom
from torch.utils.tensorboard import SummaryWriter
from statsmodels.stats.multitest import fdrcorrection
from torch_geometric.nn import Node2Vec
import gc
import matplotlib.pyplot as plt
from cuml.preprocessing import normalize, StandardScaler
import tqdm
from IPython.display import display, clear_output
from sklearn.metrics import (accuracy_score,roc_auc_score,classification_report,precision_score,recall_score,
f1_score,roc_curve,precision_recall_curve,confusion_matrix,average_precision_score,fbeta_score)
%load_ext cuml.accel


The cuml.accel extension is already loaded. To reload it, use:
  %reload_ext cuml.accel


In [2]:
G=nx.read_graphml('/home/bionik/AI_ML/FDA_Project/Data/GO_enrichment/GO_ontology.graphml')
id2node={i:u for i,(u,v) in enumerate(G.nodes(data=True))}
node2id={u:i for i,(u,v) in enumerate(G.nodes(data=True))}

G_reindexed = nx.relabel_nodes(G, node2id)
edges=[]
for u, v ,data in G_reindexed.edges(data=True):
    edges.append([u,v])
edge_index=np.array(edges)
edge_index=edge_index.transpose()
edge_index=torch.tensor(edge_index)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
node2vec=Node2Vec(edge_index=edge_index,embedding_dim=64, sparse=True,
                 walk_length=80,context_size=10,walks_per_node=8, p=0.85,q=1,num_negative_samples=3)
loader=node2vec.loader(batch_size=128,shuffle=True)
optimizer=torch.optim.SparseAdam(list(node2vec.parameters()),lr=0.01)
node2vec=node2vec.to(device)

In [ ]:
epoch=8
losses=[]
for i in range(1,epoch+1):
    x=train_model_node2vec()
    print(x)
    losses.append(x)

In [ ]:
def train_model_node2vec():
    total_loss_per_epoch=0
    node2vec.train()
    for pos,neg in loader:
        optimizer.zero_grad()
        pos=pos.to(device)
        neg=neg.to(device)
        loss=node2vec.loss(pos,neg)
        loss.backward()
        optimizer.step()
        total_loss_per_epoch+=loss.item()
        del pos,neg
        torch.cuda.empty_cache()
        gc.collect()
    return total_loss_per_epoch/len(loader)


In [ ]:
node2vec.load_state_dict(state_dict)
Z=np.array(node2vec().detach().cpu())
Z = Z.astype('float32')
norm_Z=normalize(Z,norm='l2',axis=1)

for i,(k,v) in enumerate(G_reindexed.nodes(data=True)):   # for all GO in togherte     for i,(k,v) in enumerate(id2node.items()):
    value=norm_Z[i]
    key=id2node[k]
    embedding_GO[key]=value
embedding_Go_id={node2id.get(i):v for i,v in embedding_GO.items()}
embedding_Go_id

In [ ]:
def get_embeddings_gene(annotation_of_gene, embedding_matrix,node2id,gene='NUDT4B', concat=False):
    ## genegene2ass_all {'NUDT4B': {'GO:0000298',...} dict with value is set
    ids=[] ## make sure the ids in node2id is not coming as None
    ass=list(annotation_of_gene)
    for i in ass:
        k=node2id.get(i)
        if k:
            ids.append(k)
    emb=np.sum(embedding_matrix[ids],axis=0)
    if concat:
        emb=features_x[ids].flatten()
        return emb
    return emb/emb.shape[0]


In [ ]:
prot_emb={}
for i,v in gene2ass_all_ncbi.items():
    if v:
        prot_emb[sys2gene[i]]=get_embeddings_gene(annotation_of_gene=v, embedding_matrix=embedding_GO,gene=i, concat=False,node2id=GO_embeddings_GO_2_id)

From this start the file running

In [123]:
genes_in_pathway=defaultdict(set)

list_of_all_graphml=[]
for i in os.listdir('/home/bionik/AI_ML/FDA_Project/Data/Graphs'):
    list_of_all_graphml.append(str(i).removesuffix('.graphml').lower())

with open('/home/bionik/AI_ML/FDA_Project/Data/final_data/gmts/ReactomePathways.gmt','r') as f:
    for lines in f:
        key=lines.split('\t')[0].split('_')[0].lower()
        genes=lines.strip('\n').split('\t')[2:]
        genes_in_pathway[key].update(genes)
genes_in_pathway=dict(genes_in_pathway)

embedding_GO=np.load('/home/bionik/AI_ML/FDA_Project/Data/New/GO_embeddings.npy')

with open('/home/bionik/AI_ML/FDA_Project/Data/New/GO_embeddings_ids.pkl', 'rb') as file:
    GO_embeddings_ids2node = pickle.load(file)

GO_embeddings_GO_2_id={i:v for v,i in GO_embeddings_ids2node.items()}

with open('/home/bionik/AI_ML/FDA_Project/Data/New/gene2ass_all_ncbi.pkl', 'rb') as file:
    gene2ass_all_ncbi = pickle.load(file)

with open('/home/bionik/AI_ML/FDA_Project/Data/New/synonmys_of_genes.pkl', 'rb') as file:
    sys2gene = pickle.load(file)

with open('/home/bionik/AI_ML/FDA_Project/Data/New/Protein_embeddings.pkl', 'rb') as file:
    prot_emb = pickle.load(file)
    
with open('/home/bionik/AI_ML/FDA_Project/Data/New/disease_target_aff_genes.pkl', 'rb') as file:
    disease_target_aff_genes = pickle.load(file)

with open('/home/bionik/AI_ML/FDA_Project/Data/New/disease_involment_genes.pkl', 'rb') as file:
    disease_involment_genes = pickle.load(file)

with open('/home/bionik/AI_ML/FDA_Project/Data/New/all_targets_ori_and_CHEMBL.pkl', 'rb') as file:
    disease2target = pickle.load(file)

with open('/home/bionik/AI_ML/FDA_Project/Data/New/pathway_enrichment_graphs.pkl', 'rb') as file:
    pathway_enrichment = pickle.load(file)

In [120]:

def make_pathway_union(lists_of_pathways,path='/home/bionik/AI_ML/FDA_Project/Data/Graphs'):
    list_=[]
    ultra_mapping={}
    for idx,pathway in enumerate(lists_of_pathways):
        pathway1=pathway.replace('/','_')
        idx1=nx.read_graphml(f"{path}/{pathway1}.graphml")
        list_.append(idx1)
        mapping = {f"{pathway}{node}":node for node,data in idx1.nodes(data=True) if data.get('label')!='Small_molecule'}
        ultra_mapping.update(mapping)
    G=nx.union_all(list_,rename=lists_of_pathways)
    return nx.relabel.relabel_nodes(G,ultra_mapping)



def pathway_enrichment(
    gene_list_of_interest,
    pathways,
    background_genes=None,
    min_overlap=1,
    fdr_threshold=0.01):
    
    # Convert to sets
    gene_list_of_interest = set(gene_list_of_interest)
    
    if background_genes is None:
        background_genes = set().union(*pathways.values()) | gene_list_of_interest
    else:
        background_genes = set(background_genes)
    
    N = len(background_genes)  # background size
    n = len(gene_list_of_interest)  # query size
    
    results = []
    for pathway_name, pathway_genes in pathways.items():
        pathway_genes = set(pathway_genes)
        K = len(pathway_genes)  # pathway size
        overlapping_genes = gene_list_of_interest & pathway_genes
        k = len(overlapping_genes)  # observed overlap
        
        if k < min_overlap:
            continue
        
        # Hypergeometric test
        p_value = hypergeom.sf(k - 1, N, K, n)
        
        # Expected overlap under null model
        expected = (K / N) * n
        
        # Enrichment metrics
        enrichment_ratio = (k / expected) if expected > 0 else np.nan
        odds_ratio = (k / n) / (K / N) if K > 0 else np.nan
        
        results.append({
            'Pathway': pathway_name,
            'Overlap_Count': k,
            'Pathway_Size': K,
            'Gene_List_Size': n,
            'Background_Size': N,
            'P_Value': p_value,
            'Expected_Overlap': expected,
            'Enrichment_Ratio': enrichment_ratio,
            'Odds_Ratio': odds_ratio,
            '-log10(P)': -np.log10(p_value + 1e-300),  # avoid log(0)
            'Overlapping_Genes': sorted(overlapping_genes)            
        })
    
    df = pd.DataFrame(results)
    
    if df.empty:
        return pd.DataFrame(columns=[
            'Pathway', 'Overlap_Count', 'Pathway_Size', 'Gene_List_Size', 'Background_Size',
            'P_Value', 'Adjusted_P_Value', 'Expected_Overlap', 'Enrichment_Ratio', 'Odds_Ratio', '-log10(P)', 'Overlapping_Genes'
        ])
    
    # FDR correction
    df['Adjusted_P_Value'] = fdrcorrection(df['P_Value'])[1]
    
    # Filter by FDR threshold
    df = df[df['Adjusted_P_Value'] <= fdr_threshold]
    
    return df.sort_values(by='Adjusted_P_Value').reset_index(drop=True)



def graph_generation(disease,background_genes,pathways,gene_in_disease,target,min_overlap,fdr_threshold):

    results = pathway_enrichment(gene_in_disease,pathways,background_genes=background_genes,
                                 min_overlap=2,fdr_threshold=0.05)
    df=pd.DataFrame(results) 
    enriched_pathways=df['Pathway'].to_list()
    if enriched_pathways:
        G=make_pathway_union(enriched_pathways)
        return G,disease,target,gene_in_disease,enriched_pathways
    return None ,None,None,None,None

In [121]:

protienn=set(list(sys2gene.keys())+list(sys2gene.values()))
pathways={i:v for i, v in genes_in_pathway.items() for V in v if V in protienn and i in list_of_all_graphml}
background_genes = set(sys2gene[V] for i in genes_in_pathway.values() for V in i if V in protienn)

results=[]
processed_disease=[]
for i,v in disease_target_aff_genes.items(): 
    for I in v.keys():
        if i in processed_disease:
            continue
        processed_disease.append(i)
        graph,disease,target,genes,enrichmed=graph_generation(disease=i,background_genes=background_genes,
                                                    pathways=pathways,gene_in_disease=v[I],target=I,min_overlap=1,fdr_threshold=0.10)
        
        if graph:
            results.append({
                'Graph': graph,
                'disease': disease ,   
                'pathway':enrichmed
            })

In [117]:

def preprocess_graph(G,prot_emb,sys2gene):
    protienn=set(list(sys2gene.keys())+list(sys2gene.values()))
    
    for node, attr in G.nodes(data=True):
        if not attr:
            G.nodes[node]['label']='NA'
        if node in protienn:
            G.nodes[node]['label']='Protein'
    G=nx.relabel_nodes(G,sys2gene)

    for i,v,e in G.edges(data=True):
        if not e.get('label'):
            G[i][v]['label']='NA'
            
    protein_mask=[True if v.get('label')=='Protein' else False for i,v in G.nodes(data=True)]  
    protein_mask=torch.tensor(np.array(protein_mask))
    
    return G,protein_mask
    
def get_positional_enc(G,enc_dim):
    laplace_matrix=nx.directed_laplacian_matrix(G,walk_type='random')
    eigvals, eigvecs = eigsh(laplace_matrix)
    x=eigvecs[:,1:enc_dim+1]
    id_sorted= x[:,:1].flatten().argsort()
    return x,id_sorted

def generate_graph_feature(G,Protein_embeb_dict,pos_enc_dim):
    graph_node_features=[]
    for i,v in G.nodes(data=True):
        if i in Protein_embeb_dict.keys():
            graph_node_features.append(Protein_embeb_dict[i])
        else:
            x=np.zeros(64,dtype=np.float32)
            graph_node_features.append(x)
    x,ids2=get_positional_enc(G,enc_dim=pos_enc_dim) 
    graph_node_features=np.array(graph_node_features)
    graph_node_features=np.concat((graph_node_features,x),axis=1,dtype=np.float32)
    return graph_node_features

def generate_graph_data(G,disease_involved,pos_enc,prot_embs,edge_mapping,target,sys2gene):

    # mapping: {'activates': 0, 'complexed': 1, 'inhibites': 2, 'reacts': 3, 'self': 4}
    G,protein_mask=preprocess_graph(G,prot_emb,sys2gene)
    x_feature=generate_graph_feature(G,prot_embs,pos_enc)
    disease_involved_mask=np.isin(np.array(list(G.nodes())),np.array(disease_involved)).reshape(-1,1)
    attenion_on_disease_genes=torch.tensor(np.where(disease_involved_mask,2,1))
    y=torch.tensor(np.isin(list(G.nodes()),np.array(target)))
    label=torch.tensor(torch.where(y,1,0),dtype=torch.long)
    node2id={u:i for i,(u,v) in enumerate(G.nodes(data=True))}
    G_reindexed = nx.relabel_nodes(G, node2id)
    
    edges=[]
    types=[]
    for u, v ,data in G_reindexed.edges(data=True):
        edges.append([u,v])
        types.append(data.get('label'))
        
    edge_index=np.array(edges)
    edge_index=edge_index.T
    edge_index=torch.tensor(edge_index) 
    types_relations=np.array(types)
    edge_types_for_scaling=torch.tensor(np.where(types_relations=='activates',2.0,np.where(types_relations=='inhibites',0.5,1)))
    
    edge_index_looped, _ = add_self_loops(edge_index, num_nodes=x_feature.shape[0])
    row, col = edge_index_looped
    deg = degree(col, x_feature.shape[0])
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    norm=torch.tensor(norm,dtype=torch.float32)
    remainin=x_feature.shape[0]
    rem_for_scaling=torch.ones(remainin,dtype=torch.float32)
    types_relations=np.concat((types_relations,np.array(['self']*remainin)),axis=0)
    edge_types_for_scaling=torch.tensor(torch.concat((edge_types_for_scaling,rem_for_scaling),dim=-1),dtype=torch.float32)
    types_relations=torch.tensor([edge_mapping[i] for i in types_relations],dtype=torch.long)

    data=Data(x=torch.tensor(x_feature),edge_index=edge_index_looped,node_deg_norm=norm,
             types_of_edges=types_relations,types_message_scaling=edge_types_for_scaling,
              label=label,disease_genes=attenion_on_disease_genes,protein_mask=protein_mask)


    return data
    

In [ ]:
mapping={'activates': 0, 'complexed': 1, 'inhibites': 2, 'reacts': 3, 'self': 4}

graph_list=[]
for i in pathway_enrichment:   ## of enriched pathway data with disease, graph and pathway affectde
    target_list=list(disease_target_aff_genes[i['disease']].keys())
    data1=generate_graph_data(G=i['Graph'],disease_involved=disease_target_aff_genes[i['disease']][target_list[0]],
                   pos_enc=4,prot_embs=prot_emb,edge_mapping=mapping,target=target_list,sys2gene=sys2gene)
    data1.name=i['disease']
    graph_list.append(data1)

torch.save(graph_list,'/home/bionik/AI_ML/FDA_Project/Data/New/all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt')

In [2]:
class Custom_GCNConv2(MessagePassing):
    def __init__(self, in_channels, out_channels,number_of_edges):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.bias = Parameter(torch.empty(out_channels))
        self.transform_=torch.nn.Parameter(torch.randn(number_of_edges,in_channels,out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        self.bias.data.zero_()

    def forward(self, x, edge_index,
                edge_type_scaling, norm,edge_type):

        out = self.propagate(edge_index=edge_index, x=x,
                             norm=norm ,edge_type_scaling=edge_type_scaling,edge_type=edge_type)

        return out + self.bias

    def message(self, x_j, norm,edge_type_scaling, edge_type):# shape of norm is same as edge index after the self loop
        Weights=self.transform_[edge_type]
        transformed_x_j=torch.bmm(x_j.unsqueeze(1),Weights).squeeze(1)
        del Weights
        x_j=(norm.view(-1, 1) * transformed_x_j  * edge_type_scaling.view(-1,1))
        x_j=torch.layer_norm(x_j,normalized_shape=x_j.shape)
        return  x_j

class multi_attention_transformer(torch.nn.Module):
    def __init__(self,Feed_hidden_transfomer=64,feature_dim_input=64,heads=0,drop_out=0.2,concat=True,Type='Self'):
        super(multi_attention_transformer,self).__init__()
        self.heads= heads if heads else None
        self.drop_out=drop_out
        self.feature_dim_input=feature_dim_input
        self.linear_T_dim_for_attention=feature_dim_input*heads if heads else feature_dim_input
        self.concat=concat
        self.Type=Type
        self.Q=torch.nn.Linear(self.feature_dim_input,self.linear_T_dim_for_attention)
        self.K=torch.nn.Linear(self.feature_dim_input,self.linear_T_dim_for_attention)
        self.V=torch.nn.Linear(self.feature_dim_input,self.linear_T_dim_for_attention)
        self.Linear1=torch.nn.Linear(feature_dim_input,Feed_hidden_transfomer)
        self.Linear2=torch.nn.Linear(Feed_hidden_transfomer,feature_dim_input)
        if self.heads:
            self.heads=heads
            self.head_dim=self.linear_T_dim_for_attention//self.heads
            self.Linear_output=torch.nn.Linear(self.linear_T_dim_for_attention,self.feature_dim_input)
        self.reset_parameter()


    def reset_parameter(self):
        self.Q.reset_parameters()
        self.K.reset_parameters()
        self.V.reset_parameters()
        if self.heads:
            self.Linear_output.reset_parameters()

    def attention_QKV(self,Q,K,V,attention_mask):
        feature_dim=Q.size(-1)
        Q_K=torch.matmul(Q,K.transpose(-2,-1))/torch.tensor(feature_dim**0.5)
        weights=F.softmax(Q_K,dim=-1)
        weights=F.dropout(weights,p=self.drop_out)
        output_attented=torch.matmul(weights,V)
        return output_attented, weights

    def forward(self,x1,x2,attention_mask):
        if self.Type=='Cross':
            Q=self.Q(x1)
            K=self.K(x2)
            V=self.V(x2)
        if self.Type=='Self':
            Q=self.Q(x1)
            K=self.K(x1)
            V=self.V(x1)

        if self.heads:
            self.input_size=x1.size(0)
            #print(f"thisis a multihead attention{Q.shape}")
            Q=Q.view(self.input_size,self.heads,self.head_dim).transpose(0,1)   # ## divides 1 input with 64 dim to 4 with 16 dim 
            K=K.view(self.input_size,self.heads,self.head_dim).transpose(0,1)
            V=V.view(self.input_size,self.heads,self.head_dim).transpose(0,1)
            '''B, N, D = x1.shape  if the batch size is more than 1
            Q = Q.view(B, N, self.heads, self.head_dim).transpose(1, 2)  # (B, heads, N, head_dim)
            K = K.view(B, N, self.heads, self.head_dim).transpose(1, 2)
            V = V.view(B, N, self.heads, self.head_dim).transpose(1, 2)'''
            out,wei=self.attention_QKV(Q,K,V,attention_mask)
            if self.concat:
                out=out.transpose(0,1).contiguous().view(self.input_size,self.linear_T_dim_for_attention)
                '''out = out.transpose(1, 2).contiguous().view(B, N, self.linear_T_dim_for_attention)  if batch size is more than 1'''
                attented_output=self.Linear_output(out)
            else:
                attented_output=out.mean(dim=0)
        else:
            attented_output, wei=self.attention_QKV(Q,K,V,attention_mask)

        attented_output=F.dropout(attented_output,p=self.drop_out,training=self.training)
        attented_output=F.layer_norm(attented_output+x1,normalized_shape=attented_output.shape)
        Feed_forward=F.dropout(F.relu(self.Linear1(attented_output)),p=self.drop_out,training=self.training)
        Feed_forward_out=self.Linear2(Feed_forward)
        Feed_forward_out=F.layer_norm(attented_output+F.dropout(Feed_forward_out,p=self.drop_out,training=self.training),normalized_shape=attented_output.shape)
        return Feed_forward_out



def focal_loss(logits,labels,gamma,alpha,epsilon):
    labels=labels.reshape(-1,1).to(dtype=torch.float32)
    if epsilon!=0:
        labels = labels * (1 - epsilon) + 0.5 * epsilon
    BCE_loss=F.binary_cross_entropy_with_logits(logits,labels)
    prob=torch.sigmoid(logits)
    p_t=prob*labels+(1-prob)*(1-labels)
    focal_weight=(1-p_t)**gamma
    alpha_t=alpha*labels+(1-alpha)*(1-labels)
    loss=alpha_t*BCE_loss* focal_weight
    return loss.mean()


In [3]:
class diff_model_final_model(torch.nn.Module):
    def __init__(self,input_layer,pos_enc,rels,conv_dim,output_layer,heads,feed_forward_transformer,drop_out,hidden_last_layer):
        super(diff_model_final_model,self).__init__()
        self.dropout=drop_out
        self.hidden_last_layer=hidden_last_layer
        self.input_layer=input_layer
        self.real_input=input_layer-pos_enc+2
        self.pos_enc=pos_enc
        self.conv_dim=conv_dim
        self.relations=rels
        self.output_layer=output_layer
        self.input_sage=int(input_layer-pos_enc)
        self.heads=heads
        self.feed_forward_transformer=feed_forward_transformer
        self.input_tranformer=int(conv_dim+pos_enc+2)
        self.conv_disease1=Custom_GCNConv2(self.real_input,self.conv_dim,self.relations)
        self.conv_disease2=Custom_GCNConv2(self.conv_dim,self.conv_dim,self.relations)
        self.self_attention_disease=multi_attention_transformer(self.feed_forward_transformer,self.input_tranformer,self.heads,self.dropout,True,'Self')
        self.cross_attention_disease=multi_attention_transformer(self.feed_forward_transformer,self.input_tranformer,self.heads,self.dropout,True,'Cross')
        self.classifier = torch.nn.Sequential(
            Linear(self.input_tranformer*3, self.hidden_last_layer),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(self.dropout),
            Linear(self.hidden_last_layer, self.output_layer))

    def forward(self,x,edge_index,disease_genes,protein_mask,types_of_edges,norm,edge_type_scaling):
        normal_genes=torch.where(disease_genes==2,1,torch.where(disease_genes==1,2,disease_genes))
        normal_genes[~protein_mask]=1
        involved_mask_disease=torch.where(disease_genes==2,1,0)
        involved_mask_normal=torch.where(normal_genes==2,1,0)
        disease_convolution=F.dropout(F.leaky_relu(self.conv_disease1(torch.concat([x[:,:-self.pos_enc],involved_mask_disease,involved_mask_normal],dim=1),
                                                                      edge_index,edge_type_scaling,norm,types_of_edges)),p=self.dropout,training=self.training)
        disease_convolution=F.dropout(F.leaky_relu(self.conv_disease2(disease_convolution,edge_index,edge_type_scaling,norm,types_of_edges)),p=self.dropout,training=self.training)
        disease_convolution=torch.concat([involved_mask_disease,involved_mask_normal,disease_convolution,x[:,-self.pos_enc:]],dim=1)
        disease_attended=self.self_attention_disease(disease_convolution,disease_convolution,disease_genes)
        normal_attended=self.self_attention_disease(disease_convolution,disease_convolution,normal_genes)
        disease_cross_attended=self.cross_attention_disease(disease_attended,normal_attended,normal_genes) + disease_convolution
        normal_cross_attended=self.cross_attention_disease(normal_attended,disease_attended,disease_genes) + disease_convolution
        difference=disease_cross_attended*normal_cross_attended
        difference=torch.concat([difference,disease_cross_attended,normal_cross_attended],dim=1)
        out=self.classifier(difference)
        return out


Arch_1=diff_model_final_model(input_layer=68,pos_enc=4,rels=5,output_layer=1,heads=2,
                     feed_forward_transformer=32,drop_out=0.35,hidden_last_layer=32,conv_dim=16)
Arch_1

diff_model_final_model(
  (conv_disease1): Custom_GCNConv2()
  (conv_disease2): Custom_GCNConv2()
  (self_attention_disease): multi_attention_transformer(
    (Q): Linear(in_features=22, out_features=44, bias=True)
    (K): Linear(in_features=22, out_features=44, bias=True)
    (V): Linear(in_features=22, out_features=44, bias=True)
    (Linear1): Linear(in_features=22, out_features=32, bias=True)
    (Linear2): Linear(in_features=32, out_features=22, bias=True)
    (Linear_output): Linear(in_features=44, out_features=22, bias=True)
  )
  (cross_attention_disease): multi_attention_transformer(
    (Q): Linear(in_features=22, out_features=44, bias=True)
    (K): Linear(in_features=22, out_features=44, bias=True)
    (V): Linear(in_features=22, out_features=44, bias=True)
    (Linear1): Linear(in_features=22, out_features=32, bias=True)
    (Linear2): Linear(in_features=32, out_features=22, bias=True)
    (Linear_output): Linear(in_features=44, out_features=22, bias=True)
  )
  (classifi

In [6]:
def train_model(model,loader,device,gamma,alpha,Focal_loss,max_norm,epsilon):
    loss_per_epoch=0
    model.train()
    for data in tqdm.tqdm(loader):
        if sum(data.label)==0:   ## some graphs dont have targets
            continue
        optimizer.zero_grad()
        with autocast(device_type='cuda',dtype=torch.float16):
          data=data.to(device)
          model_out=model(data.x,data.edge_index,data.disease_genes,data.protein_mask,data.types_of_edges,data.node_deg_norm,data.types_message_scaling)
          if Focal_loss:
              loss=focal_loss(model_out[data.protein_mask],data.label[data.protein_mask],gamma,alpha,epsilon)
          else:
              pos_weight=sum(data.label[data.protein_mask]==0)//sum(data.label[data.protein_mask]==1)
              loss=F.binary_cross_entropy_with_logits(model_out[data.protein_mask],data.label[data.protein_mask].reshape(-1,1).to(dtype=torch.float32),
                                                      pos_weight=torch.tensor(pos_weight))
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        loss_per_epoch+=loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
        scaler.step(optimizer)
        scaler.update()
        del data,model_out
        gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    return loss_per_epoch/len(loader)

def test_model(model,loader,device,gamma,alpha,Focal_loss,threshold,epsilon):
    model.eval()
    loss_per_epoch=0
    predictions_all=[]
    labels_all=[]
    probability_all=[]
    with torch.no_grad():
        for data in tqdm.tqdm(loader):
            if sum(data.label)==0:   ## some graphs dont have targets
                continue
            with autocast(device_type='cuda',dtype=torch.float16):
              data=data.to(device)
              model_out=model(data.x,data.edge_index,data.disease_genes,data.protein_mask,data.types_of_edges,data.node_deg_norm,data.types_message_scaling)
              if Focal_loss:
                  loss=focal_loss(model_out[data.protein_mask],data.label[data.protein_mask],gamma,alpha,epsilon)
              else:
                  pos_weight=sum(data.label[data.protein_mask]==0)//sum(data.label[data.protein_mask]==1)
                  loss=F.binary_cross_entropy_with_logits(model_out[data.protein_mask],data.label[data.protein_mask].reshape(-1,1).to(dtype=torch.float32),
                                                        pos_weight=torch.tensor(pos_weight))
            loss_per_epoch+=loss.item()
            probability=F.sigmoid(model_out)
            predictions=(probability>threshold).long()
            probability_all.extend(probability[data.protein_mask].cpu())
            predictions_all.extend(predictions[data.protein_mask].cpu())
            labels_all.extend(data.label[data.protein_mask].cpu())
            del data,model_out
            gc.collect()
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    return loss_per_epoch/len(loader),np.array(labels_all),np.array(probability_all),np.array(predictions_all)


def inference_model(model,data,device,top_k,node_2_id,threshold):
    model.eval()   # node_2_id={0:"A",1:"B",2:"C"}
    with torch.no_grad():
        data=data.to(device)
        model_output=model(data.x,data.edge_index,data.types_of_edges,data.disease_genes)
        probabilities=F.sigmoid(model_output)
        predicted_class=(probabilities>threshold).long()
        sorted_prob=torch.argsort(probabilities,dim=0,descending=True)
        top_k_targets=[node_2_id[int(i)] for i in ind]
    return probabilities,top_k_targets

In [ ]:
train_data_list=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/graph_data_list_all_targets_TRAIN_graph_data_list_from_disease_target_aff_genes_enriched.pt',weights_only=False)
train_data_list=train_data_list[:2]
epsilon=0.1
threshold=0.1
num_folds = 2
gamma=2.0
alpha=0.975
Focal_loss=True
rels=5
epochs=40
max_norm=3.0
weight_decay=0.000075
drop_out=0.20
lr=0.0001
input_layer=68
conv_dim=26
output_layer=1
pos_enc=4
heads=1
feed_forward_transformer=32
hidden_last_layer=16

writer = SummaryWriter(log_dir=f'/home/bionik/TensorBoard/New_batching/')
writer.add_text('Parameters',str(f" Gamma: {gamma} \nAlpha: {alpha} \nLearning_rate: {lr} \nWeight_decay: {weight_decay} \nMax_norm: {max_norm} \n\
Dropout: {drop_out} \nEpochs: {epochs} \nFocal_loss: {Focal_loss} \nthreshold: {threshold} \ninput_dim: {input_layer} \n\
output_layer: {output_layer} \nheads: {heads} \nfeed_forward_transformer: {feed_forward_transformer} \n\
hidden_last_layer: {hidden_last_layer} \nepsilon: {epsilon} \nrelation: {rels} \npos_enc: {pos_enc}"))
writer.flush()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold, (train_idx, validate_idx) in enumerate(kf.split(X=train_data_list)):
    train_graphs = [train_data_list[i] for i in train_idx]
    val_graphs = [train_data_list[i] for i in validate_idx]
    super_graph_train = Batch.from_data_list(train_graphs)
    sampler_train=ImbalancedSampler(super_graph_train.label,input_nodes=super_graph_train.protein_mask.int().nonzero().flatten())
    train_loader = NeighborLoader(
        super_graph_train,
        input_nodes=super_graph_train.protein_mask.int().nonzero().flatten(),
        num_neighbors=[-1,-1,-1],  # Samples up to 50 neighbors for 1 hop
        batch_size=32,       # Number of seed nodes per mini-batch
        sampler=sampler_train,
        replace=False)

    super_graph_val = Batch.from_data_list(val_graphs)
    sampler_test=ImbalancedSampler(super_graph_val.label,input_nodes=super_graph_val.protein_mask.int().nonzero().flatten())
    val_loader = NeighborLoader(
        super_graph_val,
        input_nodes=super_graph_val.protein_mask.int().nonzero().flatten(),
        num_neighbors=[-1,-1,-1],  # Samples up to 50 neighbors for 1 hop
        batch_size=32,       # Number of seed nodes per mini-batch
        sampler=sampler_test,
        replace=False)
    
    
    scaler=GradScaler()
    model=diff_model_final_model(input_layer=input_layer,pos_enc=pos_enc,rels=rels,conv_dim=conv_dim,output_layer=output_layer,heads=heads,
                     feed_forward_transformer=feed_forward_transformer,drop_out=drop_out,
                     hidden_last_layer=hidden_last_layer)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.to(device)


    for i in range(1,epochs+1):
        print('Doing:',i)
        data_=train_loader([0]).to(device)
        train_loss=train_model(model,train_loader,device,gamma,alpha,Focal_loss,max_norm,epsilon)
        test_loss,label,prob,pred=test_model(model,val_loader,device,gamma,alpha,Focal_loss,threshold,epsilon)
        print('Done with :',i)
        f1_=f1_score(label,pred,zero_division=0)
        f2_=fbeta_score(label,pred,beta=2,average='binary')
        recall=recall_score(label,pred,zero_division=0)
        accuracy=accuracy_score(label,pred)
        precision=precision_score(label,pred,zero_division=0)
        roc=roc_auc_score(label,prob)
        avr_pr=average_precision_score(label,prob)
        

        writer.add_pr_curve(f'fold_{fold}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
        writer.add_scalar(f'fold_{fold}/Avg_Precision_recall', avr_pr, global_step=i)
        writer.add_scalar(f'fold_{fold}/Recall', recall, global_step=i)
        writer.add_scalar(f'fold_{fold}/ROC', roc, global_step=i)
        writer.add_scalar(f'fold_{fold}/Accuracy', accuracy, global_step=i)
        writer.add_scalar(f'fold_{fold}/Precision', precision, global_step=i)
        writer.add_scalar(f'fold_{fold}/F1 Score', f1_, global_step=i)
        writer.add_scalar(f'fold_{fold}/F2 Score', f2_, global_step=i)
        writer.add_scalars(f'fold_{fold}/Loss',{'train':train_loss,'test':test_loss},global_step=i)
        print('Done with adding scalers')
        
        prob,idx=torch.topk(torch.tensor(prob).flatten(),k=20)
        pred=pred[idx]
        label=label[idx]
        f1_=f1_score(label,pred,zero_division=0)
        f2_=fbeta_score(label,pred,beta=2,average='binary')
        recall=recall_score(label,pred,zero_division=0)
        accuracy=accuracy_score(label,pred)
        precision=precision_score(label,pred,zero_division=0)
        roc=roc_auc_score(label,prob)
        avr_pr=average_precision_score(label,prob)
        writer.add_pr_curve(f'TopK_{fold}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
        writer.add_scalar(f'TopK_{fold}/Avg_Precision_recall', avr_pr, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Recall', recall, global_step=i)
        writer.add_scalar(f'TopK_{fold}/ROC', roc, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Accuracy', accuracy, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Precision', precision, global_step=i)
        writer.add_scalar(f'TopK_{fold}/F1 Score', f1_, global_step=i)
        writer.add_scalar(f'TopK_{fold}/F2 Score', f2_, global_step=i)
        del label,prob,pred, train_loss,test_loss


        for name,para in model.named_parameters():
            writer.add_histogram(f"fold_{fold}/Param_{name}_weights",para,i)

        if fold==0 and i==1:
            writer.add_text('Model Architecture', str(model))
            writer.add_graph(model,(data_.x,data_.edge_index,data_.disease_genes,data_.protein_mask,
                                   data_.types_of_edges,data_.node_deg_norm,data_.types_message_scaling))
            del data_
        writer.flush()


writer.flush()
writer.close()


Doing: 1


100%|██████████| 94/94 [00:27<00:00,  3.39it/s]


Done with : 1
Done with adding scalers


/tmp/ipykernel_53582/379340046.py:101: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  writer.add_pr_curve(f'TopK_{fold}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
/tmp/ipykernel_53582/3421821254.py:57: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  Q_K=torch.matmul(Q,K.transpose(-2,-1))/torch.tensor(feature_dim**0.5)
/tmp/ipykernel_53582/3421821254.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.det

Doing: 2


 18%|█▊        | 23/130 [00:06<00:31,  3.43it/s]

In [ ]:

train_data_list=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/graph_data_list_all_targets_TRAIN_graph_data_list_from_disease_target_aff_genes_enriched.pt',weights_only=False)
epsilon=0.1
threshold=0.1
num_folds = 2
gamma=2.0
alpha=0.975
Focal_loss=True
rels=5
epochs=40
max_norm=3.0
weight_decay=0.000075
drop_out=0.20
lr=0.0001
input_layer=68
conv_dim=26
output_layer=1
pos_enc=4
heads=1
feed_forward_transformer=32
hidden_last_layer=16

writer = SummaryWriter(log_dir=f'/home/bionik/TensorBoard/New_batching/')
writer.add_text('Parameters',str(f" Gamma: {gamma} \nAlpha: {alpha} \nLearning_rate: {lr} \nWeight_decay: {weight_decay} \nMax_norm: {max_norm} \n\
Dropout: {drop_out} \nEpochs: {epochs} \nFocal_loss: {Focal_loss} \nthreshold: {threshold} \ninput_dim: {input_layer} \n\
output_layer: {output_layer} \nheads: {heads} \nfeed_forward_transformer: {feed_forward_transformer} \n\
hidden_last_layer: {hidden_last_layer} \nepsilon: {epsilon} \nrelation: {rels} \npos_enc: {pos_enc}"))
writer.flush()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold, (train_idx, validate_idx) in enumerate(kf.split(X=train_data_list)):
    train_graphs = [train_data_list[i] for i in train_idx]
    val_graphs = [train_data_list[i] for i in validate_idx]
    train_loader = torch_geometric.loader.DataLoader(train_graphs, batch_size=1, shuffle=True)
    val_loader = torch_geometric.loader.DataLoader(val_graphs, batch_size=1, shuffle=False)
    
    
    scaler=GradScaler()
    model=diff_model_final_model(input_layer=input_layer,pos_enc=pos_enc,rels=rels,conv_dim=conv_dim,output_layer=output_layer,heads=heads,
                     feed_forward_transformer=feed_forward_transformer,drop_out=drop_out,
                     hidden_last_layer=hidden_last_layer)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.to(device)


    for i in range(1,epochs+1):
        print('Doing:',i)
        train_loss=train_model(model,train_loader,device,gamma,alpha,Focal_loss,max_norm,epsilon)
        test_loss,label,prob,pred=test_model(model,val_loader,device,gamma,alpha,Focal_loss,threshold,epsilon)
        print('Done with :',i)
        f1_=f1_score(label,pred,zero_division=0)
        f2_=fbeta_score(label,pred,beta=2,average='binary')
        recall=recall_score(label,pred,zero_division=0)
        accuracy=accuracy_score(label,pred)
        precision=precision_score(label,pred,zero_division=0)
        roc=roc_auc_score(label,prob)
        avr_pr=average_precision_score(label,prob)
        

        writer.add_pr_curve(f'fold_{fold}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
        writer.add_scalar(f'fold_{fold}/Avg_Precision_recall', avr_pr, global_step=i)
        writer.add_scalar(f'fold_{fold}/Recall', recall, global_step=i)
        writer.add_scalar(f'fold_{fold}/ROC', roc, global_step=i)
        writer.add_scalar(f'fold_{fold}/Accuracy', accuracy, global_step=i)
        writer.add_scalar(f'fold_{fold}/Precision', precision, global_step=i)
        writer.add_scalar(f'fold_{fold}/F1 Score', f1_, global_step=i)
        writer.add_scalar(f'fold_{fold}/F2 Score', f2_, global_step=i)
        writer.add_scalars(f'fold_{fold}/Loss',{'train':train_loss,'test':test_loss},global_step=i)
        print('Done with adding scalers')
        
        prob,idx=torch.topk(torch.tensor(prob).flatten(),k=20)
        pred=pred[idx]
        label=label[idx]
        f1_=f1_score(label,pred,zero_division=0)
        f2_=fbeta_score(label,pred,beta=2,average='binary')
        recall=recall_score(label,pred,zero_division=0)
        accuracy=accuracy_score(label,pred)
        precision=precision_score(label,pred,zero_division=0)
        roc=roc_auc_score(label,prob)
        avr_pr=average_precision_score(label,prob)
        writer.add_pr_curve(f'TopK_{fold}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
        writer.add_scalar(f'TopK_{fold}/Avg_Precision_recall', avr_pr, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Recall', recall, global_step=i)
        writer.add_scalar(f'TopK_{fold}/ROC', roc, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Accuracy', accuracy, global_step=i)
        writer.add_scalar(f'TopK_{fold}/Precision', precision, global_step=i)
        writer.add_scalar(f'TopK_{fold}/F1 Score', f1_, global_step=i)
        writer.add_scalar(f'TopK_{fold}/F2 Score', f2_, global_step=i)
        del label,prob,pred, train_loss,test_loss
        

        for name,para in model.named_parameters():
            writer.add_histogram(f"fold_{fold}/Param_{name}_weights",para,i)

        if fold==0 and i==1:
            writer.add_text('Model Architecture', str(model))
            writer.add_graph(model,(data_.x,data_.edge_index,data_.disease_genes,data_.protein_mask,
                                   data_.types_of_edges,data_.node_deg_norm,data_.types_message_scaling))
            del data_
        writer.flush()


writer.flush()
writer.close()


In [2]:
%load_ext tensorboard
%tensorboard --logdir /home/bionik/TensorBoard/New_batching/

In [ ]:
label=[]
prob=[]
model.eval()
with torch.no_grad():
    for data in test_loader:
        data=data.to(device)
        model_out=model(data.x,data.edge_index,data.disease_genes,data.protein_mask,data.types_of_edges,data.node_deg_norm,data.types_message_scaling)
        probab=F.sigmoid(model_out)
        labels=data.label[data.protein_mask].reshape(-1,1)
        label.extend(labels.cpu().flatten())
        prob.extend(probab.cpu().flatten())

thresholds = arange(0, 0.3, 0.05)
f1_scores=[]
for t in thresholds:
    prediction=(prob>t).int()
    f1_=f1_score(label,prediction)
    f1_scores.append(f1_)

best_threshold=thresholds[argmax(scores)]

precision, recall, thresholds = precision_recall_curve(label, prob)
fscore = (2 * precision * recall) / (precision + recall)
best_threshold=thresholds[argmax(fscore)]

AFTER ADDING SUBGRAPH

In [ ]:
import pickle
with open('/home/bionik/AI_ML/FDA_Project/Data/New/subgraph_all_pathway_enrichment_graphs.pkl', 'rb') as file:
    pathway_enrichment = pickle.load(file)
    
with open('/home/bionik/AI_ML/FDA_Project/Data/New/Protein_embeddings.pkl', 'rb') as file:
    prot_emb = pickle.load(file)
    
with open('/home/bionik/AI_ML/FDA_Project/Data/New/pathways_for_whole_G.pkl', 'rb') as file:
    gra = pickle.load(file)
    


In [ ]:
import networkx as nx
Whole_G=make_pathway_union(gra)

In [ ]:
Whole_G=nx.subgraph(Whole_G,nbunch=list(nx.weakly_connected_components(Whole_G))[0])

In [ ]:
nx.write_graphml(Whole_G,'/home/bionik/AI_ML/FDA_Project/Data/Graphs/Whole.graphml')

In [ ]:
Whole_G=nx.read_graphml('/home/bionik/AI_ML/FDA_Project/Data/Graphs/Whole.graphml')


In [ ]:
edge_mapping={'activates': 0, 'complexed': 1, 'inhibites': 2, 'reacts': 3, 'self': 4}
protienn=set(prot_emb.keys())

Whole_G,protein_mask,id_2_node=preprocess_graph(Whole_G,protienn)
x_feature=generate_graph_feature(Whole_G,prot_emb,4)
nodes=list(Whole_G.nodes())
node2id={i:v for v,i in id_2_node.items()}
G_reindexed = nx.relabel_nodes(Whole_G, node2id)
edges=[]
types=[]
for u, v ,data in G_reindexed.edges(data=True):
    edges.append([u,v])
    types.append(data.get('label'))
    
edge_index=np.array(edges)
edge_index=edge_index.T
edge_index=torch.tensor(edge_index) 
types_relations=np.array(types)
edge_types_for_scaling=torch.tensor(np.where(types_relations=='activates',2.0,np.where(types_relations=='inhibites',0.5,1)))

edge_index_looped, _ = add_self_loops(edge_index, num_nodes=G_reindexed.number_of_nodes())
row, col = edge_index_looped
deg = degree(col, x_feature.shape[0])
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
norm=torch.tensor(norm,dtype=torch.float32)
remainin=G_reindexed.number_of_nodes()
rem_for_scaling=torch.ones(remainin,dtype=torch.float32)
types_relations=np.concat((types_relations,np.array(['self']*remainin)),axis=0)
edge_types_for_scaling=torch.tensor(torch.concat((edge_types_for_scaling,rem_for_scaling),dim=-1),dtype=torch.float32)
types_relations=torch.tensor([edge_mapping[i] for i in types_relations],dtype=torch.long)

print('done')
graph_list=[]
for i in pathway_enrichment:
    disease_involved_mask=np.isin(np.array(nodes),np.array(list(i['Genes']))).reshape(-1,1)
    attenion_on_disease_genes=torch.tensor(np.where(disease_involved_mask,2,1))
    y=torch.tensor(np.isin(np.array(nodes),np.array(list(i['Targets']))))
    label=torch.tensor(torch.where(y,1,0),dtype=torch.long)
    data=Data(label=label,disease_genes=attenion_on_disease_genes,name=i['name'])
    graph_list.append(data)

graph_data=Data(x=torch.tensor(x_feature),edge_index_looped=edge_index_looped,edge_index=edge_index,protein_mask=protein_mask,
                node_deg_norm=norm,types_of_edges=types_relations,types_message_scaling=edge_types_for_scaling,node_mapping=node2id)
torch.save(graph_data,'/home/bionik/AI_ML/FDA_Project/Data/New/feature_edgeindex.pt')

total_graph=[]
for i in graph_list:
    if i.label.sum()>=1:
        total_graph.append(i)
    
torch.save(total_graph,'/home/bionik/AI_ML/FDA_Project/Data/New/target_genes_mask.pt') ## has disease which dont have targets

In [ ]:
import torch
graph_list=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/target_genes_mask.pt',weights_only=False)

In [ ]:
import random
lables=[i.label.sum().item() for i in total_graph] # or graph_list
test_tagerts=round(sum(lables)*0.15)
test_graphs=round(len(total_graph)*0.15)
random.shuffle(graph_list)
test_data=random.choices(graph_list,k=test_graphs)
lables=[i.label.sum().item() for i in test_data]
print(sum(lables),test_tagerts)
test_name=set((i.name for i in test_data))
train_data=[i for i in total_graph if i.name not in test_name]

In [ ]:
torch.save(train_data,'/home/bionik/AI_ML/FDA_Project/Data/New/Train_all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt') 
torch.save(test_data,'/home/bionik/AI_ML/FDA_Project/Data/New/Test_all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt') 

In [ ]:
whole_data=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/Train_all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt',weights_only=False)
graph_data=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/feature_edgeindex.pt',weights_only=False)
G=nx.read_graphml('/home/bionik/AI_ML/FDA_Project/Data/Graphs/Whole.graphml')


In [ ]:
import random  ## do same for test graph data
tar=0
max=0
from torch_geometric.data import Data
dataaaa=[]
mapping_node={v:i for i,v in graph_data.node_mapping.items()}

di=set()
for I in whole_data:
    count=0
    for Y in I.label.nonzero().flatten():
        subset,edge_index,map,edge_mask=torch_geometric.utils.k_hop_subgraph(Y.item(),12,graph_data.edge_index_looped,relabel_nodes=True)
        if subset.shape[0]>18000:
            subset,edge_index,map,edge_mask=torch_geometric.utils.k_hop_subgraph(Y.item(),10,graph_data.edge_index_looped,relabel_nodes=True)
        if subset.shape[0]>18000:
            subset,edge_index,map,edge_mask=torch_geometric.utils.k_hop_subgraph(Y.item(),9,graph_data.edge_index_looped,relabel_nodes=True)
        if subset.shape[0]>18000:
            subset,edge_index,map,edge_mask=torch_geometric.utils.k_hop_subgraph(Y.item(),8,graph_data.edge_index_looped,relabel_nodes=True)
        if edge_index.shape[1]>5000 and subset.shape[0]<18000 and I.label[subset].sum()>=2 and subset.shape[0]>7500:
            count+=I.label[subset].sum()//3.5
            tar+=1
            max+=1
            Found=True
            di.add(I.name)
            print(I.name,subset.shape,mapping_node[Y.item()])
            dataaaa.append(Data(edge_mask=edge_mask,edge_index=edge_index,subset=subset,name=I.name,label=I.label[subset],disease_genes=I.disease_genes[subset]))
        

In [ ]:
torch.save(dataaaa,'/home/bionik/AI_ML/FDA_Project/Data/New/Subgraph_Train_all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt') 
# same code test data

In [ ]:
class diff_model_final_model(torch.nn.Module):
    def __init__(self,input_layer,pos_enc,rels,conv_dim,output_layer,heads,feed_forward_transformer,drop_out,hidden_last_layer):
        super(diff_model_final_model,self).__init__()
        self.dropout=drop_out
        self.hidden_last_layer=hidden_last_layer
        self.input_layer=input_layer
        self.real_input=input_layer-pos_enc+2
        self.pos_enc=pos_enc
        self.conv_dim=conv_dim
        self.relations=rels
        self.output_layer=output_layer
        self.input_sage=int(input_layer-pos_enc)
        self.heads=heads
        self.feed_forward_transformer=feed_forward_transformer
        self.input_tranformer=int(conv_dim+pos_enc+2)
        self.conv_disease1=torch_geometric.nn.RGCNConv(self.real_input,self.conv_dim,self.relations)
        self.conv_disease2=torch_geometric.nn.RGCNConv(self.conv_dim,self.conv_dim,self.relations)
        self.self_attention_disease=multi_attention_transformer(self.feed_forward_transformer,self.input_tranformer,self.heads,self.dropout,True,'Self')
        self.cross_attention_disease=multi_attention_transformer(self.feed_forward_transformer,self.input_tranformer,self.heads,self.dropout,True,'Cross')
        self.classifier = torch.nn.Sequential(
            Linear(self.input_tranformer*3, self.hidden_last_layer),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(self.dropout),
            Linear(self.hidden_last_layer, self.output_layer))

    def forward(self,x,edge_index,disease_genes,protein_mask,types_of_edges,norm,edge_type_scaling,batch):
        normal_genes=torch.where(disease_genes==2,1,torch.where(disease_genes==1,2,disease_genes))
        normal_genes[~protein_mask]=1
        involved_mask_disease=torch.where(disease_genes==2,1,0)
        involved_mask_normal=torch.where(normal_genes==2,1,0)
        disease_convolution=F.dropout(F.leaky_relu(self.conv_disease1(torch.concat([x[:,:-self.pos_enc],involved_mask_disease,involved_mask_normal],dim=1),
                                                                      edge_index,types_of_edges)),p=self.dropout,training=self.training)
        normal_convolution=F.dropout(F.leaky_relu(self.conv_disease1(torch.concat([x[:,:-self.pos_enc],involved_mask_normal,involved_mask_disease],dim=1),
                                                                      edge_index,types_of_edges)),p=self.dropout,training=self.training)
        disease_convolution=F.dropout(F.leaky_relu(self.conv_disease2(disease_convolution,edge_index,types_of_edges)),p=self.dropout,training=self.training)
        normal_convolution=F.dropout(F.leaky_relu(self.conv_disease2(normal_convolution,edge_index,types_of_edges)),p=self.dropout,training=self.training)
        disease_convolution=torch.concat([involved_mask_disease,involved_mask_normal,disease_convolution,x[:,-self.pos_enc:]],dim=1)
        normal_convolution=torch.concat([involved_mask_normal,involved_mask_disease,normal_convolution,x[:,-self.pos_enc:]],dim=1)
        disease_attended=self.self_attention_disease(disease_convolution,disease_convolution,disease_genes)
        normal_attended=self.self_attention_disease(normal_convolution,normal_convolution,normal_genes)
        disease_cross_attended=self.cross_attention_disease(disease_attended,normal_attended,normal_genes)
        normal_cross_attended=self.cross_attention_disease(normal_attended,disease_attended,disease_genes)
        disease_global_mean=torch_geometric.nn.global_mean_pool(disease_cross_attended, batch)
        normal_global_mean=torch_geometric.nn.global_mean_pool(normal_cross_attended, batch)
        normal_cross_attended=normal_cross_attended+normal_global_mean
        disease_cross_attended=disease_cross_attended+disease_global_mean
        difference=F.layer_norm(disease_cross_attended*normal_cross_attended,normalized_shape=normal_cross_attended.shape)
        difference=torch.concat([disease_attended,normal_attended,difference],dim=1)
        out=self.classifier(difference)
        return out


Arch_1=diff_model_final_model(input_layer=68,pos_enc=4,rels=5,output_layer=1,heads=2,
                     feed_forward_transformer=32,drop_out=0.35,hidden_last_layer=32,conv_dim=16)
Arch_1

In [ ]:
train_data_list=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/Subgraph_Train_all_targets_graph_data_list_from_disease_target_aff_genes_enriched.pt',weights_only=False)
graph_data=torch.load('/home/bionik/AI_ML/FDA_Project/Data/New/feature_edgeindex.pt',weights_only=False)


In [ ]:
import random
diseases=list(set([i.name for i in train_data_list]))
val_disease=list(set(random.choices(diseases,k=round(len(diseases)*0.20))))
train_disease=list(set(diseases).difference(list(val_disease)))
train_data=[i for i in train_data_list if i.name in train_disease]
val_data=[i for i in train_data_list if i.name in val_disease]
total_target_all=sum([i.label.nonzero().shape[0] for i in train_data_list])
total_target_train=sum([i.label.nonzero().shape[0] for i in train_data])
total_target_val=sum([i.label.nonzero().shape[0] for i in val_data])
print(len(train_data)/len(train_data_list),total_target_train/total_target_all)
print(len(val_data)/len(train_data_list),total_target_val/total_target_all)

In [ ]:
# load data of subgraph whole graph data is stored in graph and data and specific subset is accessed by train_data subset
def train_list(model,graph_data,train_dataaaa,device,gamma,alpha,Focal_loss,max_norm,epsilon,L1_Reg):
    diseases=list(set([i.name for i in train_dataaaa]))
    model.train()
    loss_per_epoch=0
    for i in tqdm.tqdm(train_dataaaa):
        data=Data(x=graph_data.x[i.subset],edge_index=i.edge_index,node_deg_norm=graph_data.node_deg_norm[i.edge_mask],
                types_of_edges=graph_data.types_of_edges[i.edge_mask],types_message_scaling=graph_data.types_message_scaling[i.edge_mask],
                label=i.label,disease_genes=i.disease_genes,protein_mask=graph_data.protein_mask[i.subset],batch=torch.zeros(graph_data.x[i.subset].shape[0],dtype=torch.long))
        data=data.to(device)
        model_out=model(data.x,data.edge_index,data.disease_genes,data.protein_mask,data.types_of_edges,data.node_deg_norm,data.types_message_scaling,data.batch)
        if Focal_loss:
            loss=focal_loss(model_out[data.protein_mask],data.label[data.protein_mask],gamma,alpha,epsilon)
        else:
            pos_weight=sum(data.label[data.protein_mask]==0)//sum(data.label[data.protein_mask]==1)
            loss=F.binary_cross_entropy_with_logits(model_out[data.protein_mask],data.label[data.protein_mask].reshape(-1,1).to(dtype=torch.float32),
                                                    pos_weight=torch.tensor(pos_weight))

        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = loss + L1_Reg * l1_norm
        loss_per_epoch+=loss.item()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
        scaler.step(optimizer)
        scaler.update()
        del data,model_out
        gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    random.shuffle(train_dataaaa)
    return loss_per_epoch/len(diseases)

def test_list(model,graph_data,test_dataaaa,device,gamma,alpha,Focal_loss,threshold,epsilon):
    diseases=list(set([i.name for i in test_dataaaa]))
    model.eval()
    loss_per_epoch=0
    predictions_all=[]
    labels_all=[]
    probability_all=[]
    with torch.no_grad():
        for i in tqdm.tqdm(test_dataaaa):
            data=Data(x=graph_data.x[i.subset],edge_index=i.edge_index,node_deg_norm=graph_data.node_deg_norm[i.edge_mask],
                    types_of_edges=graph_data.types_of_edges[i.edge_mask],types_message_scaling=graph_data.types_message_scaling[i.edge_mask],
                    label=i.label,disease_genes=i.disease_genes,protein_mask=graph_data.protein_mask[i.subset],batch=torch.zeros(graph_data.x[i.subset].shape[0],dtype=torch.long))
            data=data.to(device)
            model_out=model(data.x,data.edge_index,data.disease_genes,data.protein_mask,data.types_of_edges,data.node_deg_norm,data.types_message_scaling,data.batch)
            if Focal_loss:
                loss=focal_loss(model_out[data.protein_mask],data.label[data.protein_mask],gamma,alpha,epsilon)
            else:
                pos_weight=sum(data.label[data.protein_mask]==0)//sum(data.label[data.protein_mask]==1)
                loss=F.binary_cross_entropy_with_logits(model_out[data.protein_mask],data.label[data.protein_mask].reshape(-1,1).to(dtype=torch.float32),
                                                        pos_weight=torch.tensor(pos_weight))

            loss_per_epoch+=loss.item()
            probability=F.sigmoid(model_out)
            predictions=(probability>threshold).long()
            probability_all.extend(probability[data.protein_mask].cpu())
            predictions_all.extend(predictions[data.protein_mask].cpu())
            labels_all.extend(data.label[data.protein_mask].cpu())
            del data,model_out
            gc.collect()
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    return loss_per_epoch/len(diseases),np.array(labels_all),np.array(probability_all),np.array(predictions_all)

def load_model_from_checkpoint(path,model,optimizer,scheduler,scaler,device):
    model.to(device)
    checkpoint = torch.load(path,weights_only=True,map_location=device) # initialize the model, scaler, optimizer, scheduler
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])   
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    new_epoch=checkpoint['epoch']+1
    return model,optimizer,scheduler,scaler,new_epoch
    
    


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /home/bionik/TP/

In [ ]:
epsilon=0.1
threshold=0.03
num_folds = 5
gamma=1.3
alpha=0.95
Focal_loss=True
rels=5
epochs=2
L1_Reg=0.0001
max_norm=2.0
weight_decay=0.000075
drop_out=0.35
lr=0.000075
input_layer=68
conv_dim=26
output_layer=1
pos_enc=4
heads=1
feed_forward_transformer=32
hidden_last_layer=16
step_size=30
gamma_scheduler=0.50
train_from_checkpoint=True

writer = SummaryWriter(log_dir=f'/home/bionik/TP/')
writer.add_text('Parameters',str(f" Gamma: {gamma} \nAlpha: {alpha} \nLearning_rate: {lr} \nWeight_decay: {weight_decay} \nMax_norm: {max_norm} \n\
Dropout: {drop_out} \nEpochs: {epochs} \nFocal_loss: {Focal_loss} \nthreshold: {threshold} \ninput_dim: {input_layer} \n\
output_layer: {output_layer} \nheads: {heads} \nfeed_forward_transformer: {feed_forward_transformer} \n\
hidden_last_layer: {hidden_last_layer} \nepsilon: {epsilon} \nrelation: {rels} \npos_enc: {pos_enc} \nstepsize: {step_size} \n gamma_scheduler: {gamma_scheduler}"))
writer.flush()

device = torch.device('cuda')
best_recall ,best_precision= float(0),float(0)
scaler=GradScaler(device=device)
model=diff_model_final_model(input_layer=input_layer,pos_enc=pos_enc,rels=rels,conv_dim=conv_dim,output_layer=output_layer,heads=heads,
                  feed_forward_transformer=feed_forward_transformer,drop_out=drop_out,
                  hidden_last_layer=hidden_last_layer)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma_scheduler)


if train_from_checkpoint:
    model,optimizer,scheduler,scaler,new_epoch=load_model_from_checkpoint('/home/bionik/TP/epoch_model_optimizer_scheduler.pth',model,optimizer,scheduler,scaler,device)
    
init=new_epoch if train_from_checkpoint else 1
epochs=init+epochs

model.to(device)
for i in range(init,epochs):
    print('Doing:',i)
    train_loss=train_list(model,graph_data,train_data,device,gamma,alpha,Focal_loss,max_norm,epsilon,L1_Reg)
    scheduler.step()
    test_loss,label,prob,pred=test_list(model,graph_data,val_data,device,gamma,alpha,Focal_loss,threshold,epsilon)


    print('Done with :',i)
    f1_=f1_score(label,pred,zero_division=0)
    f2_=fbeta_score(label,pred,beta=2,average='binary')
    recall=recall_score(label,pred,zero_division=0)
    accuracy=accuracy_score(label,pred)
    precision=precision_score(label,pred,zero_division=0)
    roc=roc_auc_score(label,prob)
    avr_pr=average_precision_score(label,prob)


    writer.add_pr_curve(f'fold_{1}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
    writer.add_scalar(f'fold_{1}/Avg_Precision_recall', avr_pr, global_step=i)
    writer.add_scalar(f'fold_{1}/Recall', recall, global_step=i)
    writer.add_scalar(f'fold_{1}/ROC', roc, global_step=i)
    writer.add_scalar(f'fold_{1}/Accuracy', accuracy, global_step=i)
    writer.add_scalar(f'fold_{1}/Precision', precision, global_step=i)
    writer.add_scalar(f'fold_{1}/F1 Score', f1_, global_step=i)
    writer.add_scalar(f'fold_{1}/F2 Score', f2_, global_step=i)
    writer.add_scalars(f'fold_{1}/Loss',{'train':train_loss,'test':test_loss},global_step=i)
    print('Done with adding scalers')

    val,idx=torch.topk(torch.tensor(prob).flatten(),k=2)
    f1_=f1_score(label[idx],pred[idx],zero_division=0)
    f2_=fbeta_score(label[idx],pred[idx],beta=2,average='binary')
    recall=recall_score(label[idx],pred[idx],zero_division=0)
    accuracy=accuracy_score(label[idx],pred[idx])
    precision=precision_score(label[idx],pred[idx],zero_division=0)
    roc=roc_auc_score(label[idx],prob[idx])
    avr_pr=average_precision_score(label[idx],prob[idx])
    writer.add_pr_curve(f'TopK_{1}/Precision_recall_curve',np.array(label),np.array(prob).flatten(),global_step=i)
    writer.add_scalar(f'TopK_{1}/Avg_Precision_recall', avr_pr, global_step=i)
    writer.add_scalar(f'TopK_{1}/Recall', recall, global_step=i)
    writer.add_scalar(f'TopK_{1}/ROC', roc, global_step=i)
    writer.add_scalar(f'TopK_{1}/Accuracy', accuracy, global_step=i)
    writer.add_scalar(f'TopK_{1}/Precision', precision, global_step=i)
    writer.add_scalar(f'TopK_{1}/F1 Score', f1_, global_step=i)
    writer.add_scalar(f'TopK_{1}/F2 Score', f2_, global_step=i)
    
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
                'epoch':i}, '/home/bionik/TP/epoch_model_optimizer_scheduler.pth')

    if (recall >= best_recall) & (precision >= best_precision):
        best_recall = recall
        best_precision = precision
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'scaler_state_dict': scaler.state_dict(),
                    'epoch':i}, '/home/bionik/TP/Best_model_optimizer_scheduler.pth')

    del label,prob,pred, train_loss,test_loss
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()


    for name,para in model.named_parameters():
        writer.add_histogram(f"fold_{1}/Param_{name}",para,i)

    if i==1:
        data_=Data(x=graph_data.x[train_data[0].subset],edge_index=train_data[0].edge_index,node_deg_norm=graph_data.node_deg_norm[train_data[0].edge_mask],
                types_of_edges=graph_data.types_of_edges[train_data[0].edge_mask],types_message_scaling=graph_data.types_message_scaling[train_data[0].edge_mask],
                label=train_data[0].label,disease_genes=train_data[0].disease_genes,protein_mask=graph_data.protein_mask[train_data[0].subset],batch=torch.zeros(graph_data.x[train_data[0].subset].shape[0],dtype=torch.long))
        data_=data_.to(device)
        writer.add_text('Model Architecture', str(model))
        writer.add_graph(model,(data_.x,data_.edge_index,data_.disease_genes,data_.protein_mask,
                                data_.types_of_edges,data_.node_deg_norm,data_.types_message_scaling,data_.batch))
        del data_
        gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    writer.flush()


writer.flush()
writer.close()
